In [1]:
from path import path
rootdir = path(u'/home/pfernique/Desktop/PyClangLite')

In [2]:
import subprocess
popen = subprocess.Popen(['clang++', '-x', 'c++', '-v', '-E', '/dev/null'], stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
out, err = popen.communicate()
sysincludes = err.splitlines()
headers = []
flags = ['-x', 'c++', '-std=c++11', '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS']
if '#include <...> search starts here:' not in sysincludes or 'End of search list.' not in sysincludes:
    warnings.warn('System includes not computed: parsing clang command output failed', Warning)
else:
    lindex = sysincludes.index('#include <...> search starts here:')
    rindex = sysincludes.index('End of search list.')
    sysincludes = sysincludes[lindex+1:rindex]
flags.extend(['-I'+str(path(sysinclude.strip()).abspath()) for sysinclude in sysincludes])
for sysinclude in sysincludes:
    sysinclude = path(sysinclude.strip())
    clanginclude = sysinclude/'clang'
    if clanginclude.exists():
        headers.extend([sysinclude.relpathto(header) for header in clanginclude.walkfiles('*.h')])
selfcontained = rootdir/'src'/'wrapper'/'clang.h'
selfcontained.write_lines(['#include <' + str(header) + '>' for header in headers])

In [3]:
flags

['-x',
 'c++',
 '-std=c++11',
 '-D__STDC_LIMIT_MACROS',
 '-D__STDC_CONSTANT_MACROS',
 '-I/usr/include/c++/5.2.1',
 '-I/usr/include/x86_64-linux-gnu/c++/5.2.1',
 '-I/usr/include/c++/5.2.1/backward',
 '-I/usr/local/include',
 '-I/usr/local/lib/clang/3.9.0/include',
 '-I/usr/include/x86_64-linux-gnu',
 '-I/usr/include']

In [4]:
from autowig import autowig
import pickle
try:
    with open('pyclanglite.pkl', 'r') as f:
        asg = pickle.load(f)
except:
    print 'Parsing C++ code...'
    asg = autowig.AbstractSemanticGraph()
    autowig.parser.plugin = 'pyclanglite'
    autowig.parser(asg, [selfcontained], flags=flags,
                   overload='none', bootstrap=False, permissive=True)
    with open('pyclanglite.pkl', 'w') as f:
        pickle.dump(asg, f)

In [5]:
from autowig.controller import resolve_overload, suppress_forward_declarations
resolve_overload(asg, 'none')
suppress_forward_declarations(asg)
for namespace in asg.namespaces():
    namespace.boost_python_export = False

In [6]:
function = asg.nodes('.*clang::tooling::buildASTFromCodeWithArgs').pop()
function.boost_python_export = True
classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
import itertools
classes.extend(itertools.chain(*[cls.inheritors(recursive=True) for cls in classes]))

In [7]:
dependencies = asg.dependencies(function, *classes, bases=True, access='public')

['none', 'public']


In [8]:
{asg[cls._node].access for cls in dependencies}

{'none', 'public'}

In [9]:
for dependency in dependencies:
    dependency.boost_python_export = True
for inheritor in asg['class ::llvm::SmallVectorBase'].inheritors(recursive=True):
    inheritor.boost_python_export = False
for namespace in asg.namespaces():
    if namespace.globalname.startswith('_'):
        for declaration in namespace.declarations(nested=True):
            declaration.boost_python_export = True
for cls in asg.classes():
    if cls.boost_python_export:
        if cls.localname.startswith('_') or any(ancestor.localname.startswith('_') for ancestor in cls.ancestors):
            cls.boost_python_export = False
for cls in asg.classes(templated=False, specialized=None):
    cls.is_copyable = False

In [10]:
autowig.generator.plugin = 'boost_python'
autowig.generator(asg, rootdir/'src'/'wrapper'/'_pyclanglite.cpp', None, pattern='.*', closure=False)
if autowig.parser == 'libclang':
    pass

In [11]:
module = asg[rootdir/'src'/'wrapper'/'_pyclanglite.cpp']
module.write()

for export in module.exports:
    export.write()

In [13]:
print asg[rootdir/'src'/'wrapper'/'__clang_tooling_build_ast_from_code_with_args.cpp'].get_content()

#include <boost/python.hpp>
#include <//home/pfernique/Desktop/PyClangLite/src/wrapper/clang.h>

void __clang_tooling_build_ast_from_code_with_args()
{

        std::string clang_7bbff48d109853e88270b3595c663a99_name = boost::python::extract< std::string >(boost::python::scope().attr("__name__") + ".clang");
        boost::python::object clang_7bbff48d109853e88270b3595c663a99_module(boost::python::handle<  >(boost::python::borrowed(PyImport_AddModule(clang_7bbff48d109853e88270b3595c663a99_name.c_str()))));
        boost::python::scope().attr("clang") = clang_7bbff48d109853e88270b3595c663a99_module;
        boost::python::scope clang_7bbff48d109853e88270b3595c663a99_scope = clang_7bbff48d109853e88270b3595c663a99_module;
        std::string tooling_72106f10129759b3940693116b92ef77_name = boost::python::extract< std::string >(boost::python::scope().attr("__name__") + ".tooling");
        boost::python::object tooling_72106f10129759b3940693116b92ef77_module(boost::python::handle<  >(boost:

In [12]:
raise

TypeError: exceptions must be old-style classes or derived from BaseException, not NoneType

In [ ]:
len(classes), len(asg.classes(templated=None, specialized=None))

In [ ]:
asg.includes('class ::clang::PreprocessingDirective')[0].language

In [ ]:
asg['class ::clang::PreprocessingDirective'].boost_python_export.get_content()

In [ ]:
classes

In [ ]:
asg['/'].localname

In [ ]:
asg.namespaces()

In [ ]:
asg['class ::clang::Type'].inheritors(recursive=True)

In [ ]:
asg['class ::clang::Decl'].inheritors(recursive=True)

In [ ]:
asg['class ::clang::Decl'].inheritors

In [ ]:
%debug

In [ ]:
for node in asg.nodes():
    node.boost_python_export = False

node = asg.classes('^(class |union |struct |)::llvm::StringRef$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True

asg.classes('^(class |union |struct |)::clang::ASTUnit$').pop().boost_python_export = True

asg.classes('^(class |union |struct |)::clang::ASTContext$').pop().boost_python_export = True

for function in asg.functions('^::clang::ASTContext::getSourceManager'):
    if function.result_type.globalname in [prefix + '::clang::SourceManager &' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break

node = asg.classes('^(class |union |struct |)::clang::FileID$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True
asg.functions('^::clang::FileID::isInvalid').pop().boost_python_export = True

node = asg.classes('^(class |union |struct |)::clang::SourceLocation$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True

asg.classes('^(class |union |struct |)::clang::SourceManager$').pop().boost_python_export = True
for function in asg.functions('^::clang::SourceManager::getFileID'):
    if function.localname == 'getFileID':
        function.boost_python_export = True
        break
asg.functions('^::clang::SourceManager::getFilename').pop().boost_python_export = True
asg.functions('^::clang::SourceManager::getIncludeLoc').pop().boost_python_export = True

for node in asg.classes('^(class | union|struct |)::clang::([a-zA-Z]*(Decl|Type)|DeclContext)$'):
    node.boost_python_export = True

asg.enums('^(enum |)::clang::AccessSpecifier').pop().boost_python_export = True

asg.classes('^(class | union|struct |)::clang::CXXBaseSpecifier').pop().boost_python_export = True
for function in asg.functions('^::clang::CXXBaseSpecifier::getAccessSpecifier', free=False):
    if function.localname == 'getAccessSpecifier':
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::CXXBaseSpecifier::getType', free=False):
    if function.localname == 'getType':
        function.boost_python_export = True
        break

node = asg.classes('^(class | union|struct |)::clang::QualType$').pop()
node.boost_python_export = True
if libclang:
    asg._nodes[node.node]['is_abstract'] = False
    asg._nodes[node.node]['_is_copyable'] = True
asg.functions('^::clang::QualType::isLocalConstQualified', free=False).pop().boost_python_export = True
asg.functions('^::clang::QualType::isLocalVolatileQualified', free=False).pop().boost_python_export = True
asg.functions('^::clang::QualType::getTypePtrOrNull', free=False).pop().boost_python_export = True

asg.functions('^(class | union|struct |)::clang::Type::isBuiltinType', free=False).pop().boost_python_export = True
for function in asg.functions('^::clang::Type::isSpecificBuiltinType', free=False):
    if function.localname == 'isSpecificBuiltinType':
        function.boost_python_export = True
        break
asg.functions('^::clang::Type::isPointerType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isLValueReferenceType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isRValueReferenceType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getPointeeType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isStructureOrClassType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isEnumeralType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::isUnionType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getAsTagDecl', free=False).pop().boost_python_export = True
asg.enums('(enum |)::clang::Type::TypeClass').pop().boost_python_export = True
asg.functions('^::clang::Type::isBuiltinType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getUnqualifiedDesugaredType', free=False).pop().boost_python_export = True
asg.functions('^::clang::Type::getCanonicalTypeInternal', free=False).pop().boost_python_export = True
for function in asg.functions('^::clang::Type::getTypeClass', free=False):
    if function.localname == 'getTypeClass':
        function.boost_python_export = True
        break

asg.functions('::clang::ElaboratedType::getNamedType', free=False).pop().boost_python_export = True
asg.functions('::clang::ElaboratedType::desugar', free=False).pop().boost_python_export = True

asg.enums('^(enum |)::clang::BuiltinType::Kind$').pop().boost_python_export = True

asg.functions('^::clang::Decl::isImplicit', free=False).pop().boost_python_export = True
asg.functions('^::clang::Decl::getAccessUnsafe', free=False).pop().boost_python_export = True
asg.enums('(enum |)::clang::Decl::Kind').pop().boost_python_export = True
asg.functions('^::clang::Decl::getKind').pop().boost_python_export = True
asg.functions('^::clang::Decl::getLocation').pop().boost_python_export = True
asg.functions('^::clang::Decl::getASTContext').pop().boost_python_export = True
for function in asg.functions('^::clang::Decl::getDeclContext', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getDeclKind', free=False):
    if function.localname == 'getDeclKind':
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getParent', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getLexicalParent', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break
for function in asg.functions('^::clang::DeclContext::getLookupParent', free=False):
    if function.result_type.globalname in [prefix + '::clang::DeclContext *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
        break

asg.functions('^::clang::NamespaceDecl::isInline', free=False).pop().boost_python_export = True

asg.functions('^::clang::LinkageSpecDecl::getLanguage', free=False).pop().boost_python_export = True
asg.enums('(enum |)::clang::LinkageSpecDecl::LanguageIDs').pop().boost_python_export = True

asg.functions('^::clang::ValueDecl::getType').pop().boost_python_export = True

asg.functions('^::clang::FieldDecl::isMutable').pop().boost_python_export = True

for fct in asg.functions('^::clang::FunctionDecl::isDeleted'):
    if fct.localname == 'isDeleted':
        fct.boost_python_export = True
asg.functions('^::clang::FunctionDecl::getNumParams').pop().boost_python_export = True
for function in asg.functions('^::clang::FunctionDecl::getParamDecl', free=False):
    if function.result_type.globalname in [prefix + '::clang::ParmVarDecl *' for prefix in ['', 'class ']]:
        function.is_overloaded = True
        function.boost_python_export = True
for function in asg.functions('^::clang::FunctionDecl::getReturnType', free=False):
    if function.localname == 'getReturnType':
        function.boost_python_export = True
        break

for function in asg.functions('^::clang::CXXMethodDecl::isStatic', free=False):
    if function.localname == 'isStatic':
        function.boost_python_export = True
        break
asg.functions('^::clang::CXXMethodDecl::isConst', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXMethodDecl::isVolatile', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXMethodDecl::isVirtual', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXMethodDecl::isPure', free=False).pop().boost_python_export = True

asg.functions('^::clang::TagDecl::isClass', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::isStruct', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::isUnion', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::hasNameForLinkage', free=False).pop().boost_python_export = True
asg.functions('^::clang::TagDecl::getTypedefNameForAnonDecl', free=False).pop().boost_python_export = True
for function in asg.functions('::clang::TagDecl::isCompleteDefinition', free=False):
    if function.localname == 'isCompleteDefinition':
        function.boost_python_export = True
        break

asg.functions('^::clang::CXXRecordDecl::isAbstract', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXRecordDecl::getNumBases', free=False).pop().boost_python_export = True
asg.functions('^::clang::CXXRecordDecl::getNumVBases', free=False).pop().boost_python_export = True

for function in asg.functions('^::clang::CXXConstructorDecl::isCopyConstructor'):
    if len(function.parameters) == 0:
        function.boost_python_export = True
        function.asg._nodes[function.node]['is_const'] = True
for function in asg.functions('^::clang::CXXConstructorDecl::isMoveConstructor'):
    if len(function.parameters) == 0:
        function.boost_python_export = True
        function.asg._nodes[function.node]['is_const'] = True
asg.functions('^::clang::ClassTemplateDecl::isThisDeclarationADefinition', free=False).pop().boost_python_export = True

asg.functions('^::clang::ClassTemplateSpecializationDecl::getTemplateArgs', free=False).pop().boost_python_export = True
asg.functions('^::clang::ClassTemplateSpecializationDecl::isExplicitSpecialization', free=False).pop().boost_python_export = True
for function in asg.functions('^::clang::ClassTemplateSpecializationDecl::getSpecializedTemplate', free=False):
    if function.localname == 'getSpecializedTemplate':
        function.boost_python_export = True
        break
asg.classes('^(class |union |struct |)::clang::TemplateArgumentList$').pop().boost_python_export = True
asg.functions('^::clang::TemplateArgumentList::get', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgumentList::size', free=False).pop().boost_python_export = True

asg.classes('^(class |union |struct |)::clang::TemplateArgument$').pop().boost_python_export = True
asg.enums('^(enum |)::clang::TemplateArgument::ArgKind').pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getKind', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getAsType', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getAsDecl', free=False).pop().boost_python_export = True
asg.functions('^::clang::TemplateArgument::getIntegralType', free=False).pop().boost_python_export = True

asg.functions('^::clang::TypedefNameDecl::getUnderlyingType', free=False).pop().boost_python_export = True

for enum in asg.enums():
    if not enum.clean:
        for constant in enum.constants:
            constant.boost_python_export = True

decl = asg.classes('^(class |struct |)::clang::Decl$').pop()
decl.is_copyable = False
for inheritor in decl.inheritors(True):
    inheritor.is_copyable = False

asg['::clang'].boost_python_export = True
asg['::llvm'].boost_python_export = True

if libclang:
    asg['::llvm::StringRef'].header.is_primary = True
else:
    asg['class ::llvm::StringRef'].header.is_primary = True

if libclang:
    asg['::clang::FileID'].header.is_primary = True
else:
    asg['class ::clang::FileID'].header.is_primary = True

In [ ]:
from autowig import autowig